In [1]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

//anaconda/envs/python3/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
# Uncomment to get complete text output and not truncated text from data frames
pd.set_option('display.max_colwidth', -1)

In [3]:
# 2018 Files
# Train: 5009 labeled
# Test: 3532 labeled

#file_head = 'germeval2018_training.csv' # 5009 labeled
#file_testset = 'germeval2018_test.csv'

# For submission
#file_head = 'germeval2019_training_subtask12.csv' 
file_train18 = 'germeval2018.training.txt' # 5009 labeled
file_test18 = 'germeval2018.test_.txt' # 3532 labeled
file_train19 = 'germeval2019_training_subtask12.txt' # 3995 labeled (removed one problem-tweet -> 3994)
file_list = [file_train18, file_test18, file_train19] # should be length 12.536

file_testset = 'germeval2019_Testdata_Subtask12.txt' # 3031 unlabeled

In [4]:
# Dev
#df_tweets_tsh = pd.read_csv(file_head, delimiter='\t', header=None)

# Submission
df_tweets_tsh = pd.concat([pd.read_csv(f, delimiter='\t', header=None) for f in file_list ])
test = pd.read_csv(file_testset, sep='\r\n', encoding='utf-8-sig', header=None, names=['text', 'label', 'fine'])

//anaconda/envs/python3/lib/python3.5/site-packages/ipykernel/__main__.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [5]:
#f0 = pd.read_csv(file_testset, delimiter='\t', header=None)
#f1 = pd.read_csv(file_train18, delimiter='\t', header=None)
#f2 = pd.read_csv(file_test18, delimiter='\t', header=None)
#f3 = pd.read_csv(file_train19, delimiter='\t', encoding='utf-8-sig', header=None)
#len(f0), len(f1), len(f2), len(f3) # (3031, 5009, 3398, 3994)
#f3[3780:3820] # +1 # 3780 ok -> 3820 falsch -> 3789/3790 = problem

In [6]:
len(df_tweets_tsh), len(test)

(12387, 3031)

### Fix the Seed for Reproducible Results

In [7]:
import random

seed = 23
#torch.manual_seed(seed)
#torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

In [8]:
import re
def my_preprocess_text(in_text):
    username_re = re.compile(r'(^|[^@\w])@(\w{1,15})\b') # @username
    url_re = re.compile(r'http\S+') # urls
    in_text = re.sub('RT', '', in_text.rstrip()) # remove "RT"
    in_text = re.sub('\:', '', in_text.rstrip()) # remove ":"
    in_text = re.sub('\'s', '', in_text.rstrip()) # remove "'s" -> try without this
    in_text = re.sub('#', '', in_text.rstrip()) # remove "#"
    in_text = re.sub(url_re, 'xx_url', in_text.rstrip()) # replace urls with xx_url
    return re.sub(username_re, ' xx_username', in_text.rstrip()).lstrip() # replace @username with xx_username
    # TODO: replace multiple usernames in a row ?! 

In [9]:
df_tweets_tsh[0] = df_tweets_tsh[0].apply(my_preprocess_text)
df_tweets_tsh.head()

,0,1,2
0,"xx_username Liebe Corinna, wir würden dich gerne als Moderatorin für uns gewinnen! Wärst du begeisterbar?",OTHER,OTHER
1,"xx_username Sie haben ja auch Recht. Unser Tweet war etwas missverständlich. Dass das BVerfG Sachleistungen nicht ausschließt, kritisieren wir.",OTHER,OTHER
2,xx_username fröhlicher gruß aus der schönsten stadt der welt theo ⚓️,OTHER,OTHER
3,xx_username Amis hätten alles und jeden gewählt...nur Hillary wollten sie nicht und eine Fortsetzung von Obama-Politik erst recht nicht..!,OTHER,OTHER
4,xx_username kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper SPD in die Versenkung.,OFFENSE,INSULT


In [10]:
test['text'] = test['text'].apply(my_preprocess_text)
test.head()

,text,label,fine
0,xx_username xx_username Komisch das die Realitätsverweigerung immer von linken erbärmlichen Correctiv Accounts ausgeht...,NaN,NaN
1,"xx_username xx_username xx_username xx_username CDU steht seid Strauss, Kohl, Schäuble, vdLeyen für Korruption, Schwarzgeld, und dies überall...",NaN,NaN
2,"xx_username Die Gesichter, Namen, Religion der Täter würde zur Aufklärung beitragen...",NaN,NaN
3,"xx_username Wie verwirrt muss man sein um sich zu weigern die Tatsachen anzuerkennen, das dieser Hass von Ferkel importiert wurde....was bezweckt man damit sich selber so zu belügen, unverständlich",NaN,NaN
4,"xx_username xx_username Weil die AfD den Ferkelfunk abschaffen will, deshalb haben die Linken Zecken, Schmarotzer Angst für ihr kacken arbeiten zu müssen, deshalb sind die Führer treu und senden jeden Dreck, Lüge...",NaN,NaN


In [11]:
# Transform from the format text, binary, fine to
# text, other, offense for binary classification
# 1) Copy
df_tweets_tsh_new = df_tweets_tsh.copy()
# 2) Remove 3rd column
del(df_tweets_tsh_new[2])
# 3) Add 'other', 'offense' and use column names
df_tweets_tsh_new.columns = ['text', 'labels']
df_tweets_tsh_new['other'] = 0
df_tweets_tsh_new['offense'] = 0
# 4) Fill 'other' and 'offense'
mask_other = df_tweets_tsh_new.labels == 'OTHER'
mask_offense = df_tweets_tsh_new.labels == 'OFFENSE'

df_tweets_tsh_new.loc[mask_other, 'other'] = 1
df_tweets_tsh_new.loc[mask_offense, 'offense'] = 1
df_tweets_tsh_new.head()

,text,labels,other,offense
0,"xx_username Liebe Corinna, wir würden dich gerne als Moderatorin für uns gewinnen! Wärst du begeisterbar?",OTHER,1,0
1,"xx_username Sie haben ja auch Recht. Unser Tweet war etwas missverständlich. Dass das BVerfG Sachleistungen nicht ausschließt, kritisieren wir.",OTHER,1,0
2,xx_username fröhlicher gruß aus der schönsten stadt der welt theo ⚓️,OTHER,1,0
3,xx_username Amis hätten alles und jeden gewählt...nur Hillary wollten sie nicht und eine Fortsetzung von Obama-Politik erst recht nicht..!,OTHER,1,0
4,xx_username kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper SPD in die Versenkung.,OFFENSE,0,1


In [12]:
# Use all tweets for training
train_df_tweets_tsh = df_tweets_tsh_new.copy()

In [13]:
train_df_tweets_tsh.head()

,text,labels,other,offense
0,"xx_username Liebe Corinna, wir würden dich gerne als Moderatorin für uns gewinnen! Wärst du begeisterbar?",OTHER,1,0
1,"xx_username Sie haben ja auch Recht. Unser Tweet war etwas missverständlich. Dass das BVerfG Sachleistungen nicht ausschließt, kritisieren wir.",OTHER,1,0
2,xx_username fröhlicher gruß aus der schönsten stadt der welt theo ⚓️,OTHER,1,0
3,xx_username Amis hätten alles und jeden gewählt...nur Hillary wollten sie nicht und eine Fortsetzung von Obama-Politik erst recht nicht..!,OTHER,1,0
4,xx_username kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper SPD in die Versenkung.,OFFENSE,0,1


In [14]:
train_df_tweets_tsh.text[0]

0    xx_username Liebe Corinna, wir würden dich gerne als Moderatorin für uns gewinnen! Wärst du begeisterbar?                   
0    Meine Mutter hat mir erzählt, dass mein Vater einen Wahlkreiskandidaten nicht gewählt hat, weil der gegen die Homo-Ehe ist ☺
0    xx_username Hat die Polizei keine Kanone mehr ? 20 mal in die Munition laufen lassen und Ruhe ist .                         
Name: text, dtype: object

In [15]:
lens = train_df_tweets_tsh.text.str.len()
lens.mean(), lens.std(), lens.max()

(141.51126180673288, 114.77643275298111, 4617)

In [16]:
lens.hist();

In [17]:
# long_tweets = train_df_tweets_tsh.text.str.len() > 500 # ok, da diverse tokens verwendet werden
# train_df_tweets_tsh[long_tweets]

In [18]:
label_cols = ['other', 'offense']
train_df_tweets_tsh['none'] = 1-train_df_tweets_tsh[label_cols].max(axis=1)
train_df_tweets_tsh.describe()

,other,offense,none
count,12387.000000,12387.000000,12387.0
mean,0.667393,0.332607,0.0
std,0.471166,0.471166,0.0
min,0.000000,0.000000,0.0
25%,0.000000,0.000000,0.0
50%,1.000000,0.000000,0.0
75%,1.000000,1.000000,0.0
max,1.000000,1.000000,0.0


In [19]:
# None-Tokenizer works surprisingly well :o
def tokenize_dummy(s):
  return s.split(' ')

In [20]:
#from fastai import *
#from fastai.text import *

#tokenizer_fastai = Tokenizer(lang='de', n_cpus=8)

In [21]:
LEMMATIZE = False

import spacy
nlp = spacy.load('de')

def tokenize_spacy(corpus, lemma=LEMMATIZE):
  doc = nlp(corpus)
  if lemma:
    return list(str(x.lemma_) for x in doc) # lemma_ to get string instead of hash
  else:
    return list(str(x) for x in doc)

In [22]:
COMMENT = 'text'
#n = train_df_tweets_tsh.shape[0]
vec = TfidfVectorizer(analyzer='char', ngram_range=(3,6), tokenizer=tokenize_spacy,
               min_df=4, max_df=0.4, strip_accents='unicode', use_idf=True,
               smooth_idf=False, sublinear_tf=True, lowercase=True, binary=False)
trn_term_doc = vec.fit_transform(train_df_tweets_tsh[COMMENT])
test_term_doc = vec.transform(test[COMMENT])

In [23]:
trn_term_doc, test_term_doc

(<12387x142224 sparse matrix of type '<class 'numpy.float64'>'
 	with 4974562 stored elements in Compressed Sparse Row format>,
 <3031x142224 sparse matrix of type '<class 'numpy.float64'>'
 	with 1345018 stored elements in Compressed Sparse Row format>)

In [24]:
#vec.vocabulary_
#vec

In [25]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [26]:
x = trn_term_doc
test_x = test_term_doc

In [27]:
# other, offense
class_weight = {0: 2, 1: 1}

def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=14.0, dual=False, solver='saga', multi_class='auto', penalty='l2', 
                           class_weight='balanced', max_iter=500) # class_weight='balanced'
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [28]:
preds = np.zeros((len(test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train_df_tweets_tsh[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit other
fit offense


In [29]:
# All predictions for the testset file
predictions = pd.DataFrame(preds, columns = label_cols)
#predictions

In [30]:
submission = test.copy()
del(submission['fine'])
submission['text'][21]

'xx_username Genau der Piotoitsche Gelsenkirchener...wie gut das der Stempel noch frisch ist, dann kann man das toitsch sein sogar noch riechen...'

In [31]:
submission = test.copy()
submission['fine'] = 'OTHER' # dummy label for binary classification
for index,row in submission.iterrows():
  if( preds[index][1] >=0.485):
    #print('OFFENSE')
    submission['label'][index] = 'OFFENSE'
  else:
    #print('OTHER')
    submission['label'][index] = 'OTHER'
submission.head()

//anaconda/envs/python3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/envs/python3/lib/python3.5/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,text,label,fine
0,xx_username xx_username Komisch das die Realitätsverweigerung immer von linken erbärmlichen Correctiv Accounts ausgeht...,OTHER,OTHER
1,"xx_username xx_username xx_username xx_username CDU steht seid Strauss, Kohl, Schäuble, vdLeyen für Korruption, Schwarzgeld, und dies überall...",OTHER,OTHER
2,"xx_username Die Gesichter, Namen, Religion der Täter würde zur Aufklärung beitragen...",OTHER,OTHER
3,"xx_username Wie verwirrt muss man sein um sich zu weigern die Tatsachen anzuerkennen, das dieser Hass von Ferkel importiert wurde....was bezweckt man damit sich selber so zu belügen, unverständlich",OFFENSE,OTHER
4,"xx_username xx_username Weil die AfD den Ferkelfunk abschaffen will, deshalb haben die Linken Zecken, Schmarotzer Angst für ihr kacken arbeiten zu müssen, deshalb sind die Führer treu und senden jeden Dreck, Lüge...",OFFENSE,OTHER


In [32]:
# 1,1 ngrams -> 62.95 (C=40.0); lowercase=False
# 1,2 ngrams -> 62.88 (C=40.0); lowercase=False

# 1,1 ngrams -> 64.47 (C=1.0); lowercase=False

# 1,1 ngrams -> 66.41 (C=1.0); lowercase=True
# 1,2 ngrams -> 65.88 (C=1.0); lowercase=True
# 1,3 ngrams -> 65.80 (C=1.0); lowercase=True
# 1,4 ngrams -> 66.02  (C=1.0); lowercase=True
# 1,5 ngrams -> 65.87 (C=1.0); lowercase=True
# 1,6 ngrams -> 65.87 (C=1.0); lowercase=True

# 1,4 ngrams ->  65.79 (C=1.0); lowercase=True; class_weight='balanced'

# 3,3 ngrams -> 58.07

# 1,4 ngrams ->  66.02 (C=1.0); lowercase=True; 4/1.0
# 1,4 ngrams ->  64.07(C=1.0); lowercase=True; 2/0.8

# 1,1 ngrams ->  66.41 (C=1.0); lowercase=True; 4/0.8
# 1,1 ngrams ->  66.99 (C=4.0); lowercase=True; 4/0.8
# 1,1 ngrams ->  67.69 (C=4.0); lowercase=True; 3/0.8
# 1,1 ngrams ->  67.97 (C=4.0); lowercase=True; 2/0.8
# 1,1 ngrams ->  61.61 (C=4.0); lowercase=True; 1/0.8

# 1,1 ngrams ->  63.06 (C=8.0); lowercase=True; 2/0.8
# 1,1 ngrams ->  66.26 (C=1.0); lowercase=True; 2/0.8
# 1,1 ngrams ->  67.47 (C=2.0); lowercase=True; 2/0.8
# 1,1 ngrams ->  68.05 (C=3.0); lowercase=True; 2/0.8*

# 1,1 ngrams ->  61.88 (C=3.0); lowercase=True; 3/0.8
# 1,1 ngrams ->  68.00 (C=3.5); lowercase=True; 2/0.8

# 1,2 ngrams ->  66.16 (C=3.0); lowercase=True; 2/0.8
# 1,3 ngrams ->  65.93 (C=3.0); lowercase=True; 2/0.8
# 1,4 ngrams ->  65.95 (C=3.0); lowercase=True; 2/0.8

# 2,3 ngrams ->  59.88 (C=3.0); lowercase=True; 2/0.8

# character level
# 3,6 ngrams -> 70.71 (c=4.0); lowercase=True; 4/1.0
# 3,6 ngrams -> 69.03 (c=4.0); lowercase=False; 4/1.0

# 3,6 ngrams -> 71.03 (c=8.0); lowercase=True; 4/1.0
# 3,6 ngrams -> 69.17 (c=2.0); lowercase=True; 4/1.0
# 3,6 ngrams -> 70.79 (c=6.0); lowercase=True; 4/1.0
# 3,6 ngrams -> 71.56 (c=16.0); lowercase=True; 4/1.0
# 3,6 ngrams -> 71.11 (c=32.0); lowercase=True; 4/1.0
# 3,6 ngrams -> 71.04 (c=24.0); lowercase=True; 4/1.0
# 3,6 ngrams -> 71.29 (c=20.0); lowercase=True; 4/1.0
# 3,6 ngrams -> 71.39 (c=18.0); lowercase=True; 4/1.0
# 3,6 ngrams -> 71.38 (c=12.0); lowercase=True; 4/1.0
# 3,6 ngrams -> 71.71* (c=14.0); lowercase=True; 4/1.0
# 3,6 ngrams -> 71.57 (c=15.0); lowercase=True; 4/1.0
# 3,6 ngrams -> 71.45 (c=17.0); lowercase=True; 4/1.0

# 3,6 ngrams -> 71.14 (c=14.0); lowercase=True; 5/1.0
# 3,6 ngrams -> 71.23 (c=14.0); lowercase=True; 3/1.0
# 3,6 ngrams -> 71.21 (c=14.0); lowercase=True; 3/0.5
# 3,6 ngrams -> 71.57 (c=14.0); lowercase=True; 4/0.5

# 3,6 ngrams -> 69.32 (c=14.0); lowercase=True; 4/1.0; l1 regularization

# 3,10 ngrams -> 70.89 (c=14.0); lowercase=True; 4/1.0
# 3,8 ngrams ->  70.92 (c=14.0); lowercase=True; 4/1.0
# 2,6 ngrams ->  71.07 (c=14.0); lowercase=True; 4/1.0
# 2,5 ngrams ->  71.15 (c=14.0); lowercase=True; 4/1.0
# 4,7 ngrams ->  71.34 (c=14.0); lowercase=True; 4/1.0
# 5,8 ngrams ->  69.84 (c=14.0); lowercase=True; 4/1.0

# 3,6 ngrams -> 71.61 (c=14.0); lowercase=True; 4/0.4

# 1,10 ngrams -> 70.70
# 1,6 ngrams -> 70.99
# 1,40 ngrams -> 69.71

# Class labels (strangely balanced works best)
# 2/1: 70.3
# 1/2: 71.95
# balanced: 72.28*

# Cutoff (note that this should be 0.5)
# 0.5 -> 72.28
# 0.51 -> 72.14
# 0.55 -> 71.76
# 0.6 -> 71.59
# 0.4 -> 71.20

# 0.495 -> 72.47
# 0.49 -> 72.47
# 0.485 -> 72.62* (lemmatized: 72.58)
# 0.48 -> 72.44

# 0.47 -> 72.24

submission.to_csv('naive.csv', sep='\t', line_terminator='\n', header=None, index=False, encoding='utf-8-sig')

### Fine grained task

In [33]:
# Transform from the format text, binary, fine to
# text, 'other', 'offense', 'abuse', 'insult', 'profanity' for finegrained classification
# 1) Copy
#df_tweets_tsh_fine = pd.read_csv(file_head, delimiter='\t', header=None)
df_tweets_tsh_fine = pd.concat([pd.read_csv(f, delimiter='\t', header=None) for f in file_list ])

# 2) Remove 2rd column
del(df_tweets_tsh_fine[1])
# 3) Add 'other', 'offense', 'abuse', 'insult', 'profanity' and use column names
df_tweets_tsh_fine.columns = ['text', 'labels']
df_tweets_tsh_fine['other'] = 0
df_tweets_tsh_fine['offense'] = 0
df_tweets_tsh_fine['abuse'] = 0
df_tweets_tsh_fine['insult'] = 0
df_tweets_tsh_fine['profanity'] = 0

# 4) Fill 'other' and 'offense'
mask_other = df_tweets_tsh_fine.labels == 'OTHER'
mask_offense = df_tweets_tsh_fine.labels == 'OFFENSE'
mask_abuse = df_tweets_tsh_fine.labels == 'ABUSE'
mask_insult = df_tweets_tsh_fine.labels == 'INSULT'
mask_profanity = df_tweets_tsh_fine.labels == 'PROFANITY'

df_tweets_tsh_fine.loc[mask_other, 'other'] = 1
df_tweets_tsh_fine.loc[mask_offense, 'offense'] = 1
df_tweets_tsh_fine.loc[mask_abuse, 'abuse'] = 1
df_tweets_tsh_fine.loc[mask_insult, 'insult'] = 1
df_tweets_tsh_fine.loc[mask_profanity, 'profanity'] = 1

df_tweets_tsh_fine.head()

,text,labels,other,offense,abuse,insult,profanity
0,"@corinnamilborn Liebe Corinna, wir würden dich gerne als Moderatorin für uns gewinnen! Wärst du begeisterbar?",OTHER,1,0,0,0,0
1,"@Martin28a Sie haben ja auch Recht. Unser Tweet war etwas missverständlich. Dass das BVerfG Sachleistungen nicht ausschließt, kritisieren wir.",OTHER,1,0,0,0,0
2,@ahrens_theo fröhlicher gruß aus der schönsten stadt der welt theo ⚓️,OTHER,1,0,0,0,0
3,@dushanwegner Amis hätten alles und jeden gewählt...nur Hillary wollten sie nicht und eine Fortsetzung von Obama-Politik erst recht nicht..!,OTHER,1,0,0,0,0
4,@spdde kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper #SPD in die Versenkung.,INSULT,0,0,0,1,0


In [34]:
train_df_tweets_tsh_fine = df_tweets_tsh_fine.copy()

In [35]:
train_df_tweets_tsh_fine.head()

,text,labels,other,offense,abuse,insult,profanity
0,"@corinnamilborn Liebe Corinna, wir würden dich gerne als Moderatorin für uns gewinnen! Wärst du begeisterbar?",OTHER,1,0,0,0,0
1,"@Martin28a Sie haben ja auch Recht. Unser Tweet war etwas missverständlich. Dass das BVerfG Sachleistungen nicht ausschließt, kritisieren wir.",OTHER,1,0,0,0,0
2,@ahrens_theo fröhlicher gruß aus der schönsten stadt der welt theo ⚓️,OTHER,1,0,0,0,0
3,@dushanwegner Amis hätten alles und jeden gewählt...nur Hillary wollten sie nicht und eine Fortsetzung von Obama-Politik erst recht nicht..!,OTHER,1,0,0,0,0
4,@spdde kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper #SPD in die Versenkung.,INSULT,0,0,0,1,0


In [36]:
label_cols_fine = ['other', 'abuse', 'insult', 'profanity']
train_df_tweets_tsh_fine['none'] = 1-train_df_tweets_tsh_fine[label_cols_fine].max(axis=1)
train_df_tweets_tsh_fine.describe()

,other,offense,abuse,insult,profanity,none
count,12387.000000,12387.0,12387.000000,12387.000000,12387.000000,12387.0
mean,0.667393,0.0,0.182853,0.128118,0.021636,0.0
std,0.471166,0.0,0.386562,0.334235,0.145496,0.0
min,0.000000,0.0,0.000000,0.000000,0.000000,0.0
25%,0.000000,0.0,0.000000,0.000000,0.000000,0.0
50%,1.000000,0.0,0.000000,0.000000,0.000000,0.0
75%,1.000000,0.0,0.000000,0.000000,0.000000,0.0
max,1.000000,0.0,1.000000,1.000000,1.000000,0.0


In [37]:
COMMENT = 'text'
#n = train_df_tweets_tsh_fine.shape[0]
vec = TfidfVectorizer(analyzer='char', ngram_range=(3,6), tokenizer=tokenize_spacy,
               min_df=4, max_df=1.0, strip_accents='unicode', use_idf=True,
               smooth_idf=False, sublinear_tf=True, lowercase=True, binary=False)
trn_term_doc_fine = vec.fit_transform(train_df_tweets_tsh_fine[COMMENT])
test_term_doc_fine = vec.transform(test[COMMENT])

In [38]:
trn_term_doc_fine, test_term_doc_fine

(<12387x174575 sparse matrix of type '<class 'numpy.float64'>'
 	with 5742240 stored elements in Compressed Sparse Row format>,
 <3031x174575 sparse matrix of type '<class 'numpy.float64'>'
 	with 1381344 stored elements in Compressed Sparse Row format>)

In [39]:
x = trn_term_doc_fine
test_x = test_term_doc_fine

In [40]:
# other, abuse, insult, profanity
# class_weight = {}

def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=14.0, dual=False, solver='saga', multi_class='auto', penalty='l2', max_iter=500, class_weight='balanced') # class_weight='balanced'
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [41]:
preds = np.zeros((len(test), len(label_cols_fine)))

for i, j in enumerate(label_cols_fine):
    print('fit', j)
    m,r = get_mdl(train_df_tweets_tsh_fine[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit other
fit abuse
fit insult
fit profanity


//anaconda/envs/python3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [42]:
# All predictions for the testset file
predictions = pd.DataFrame(preds, columns = label_cols_fine)
#predictions

In [43]:
submission = test.copy()
#del(submission['fine'])
submission['text'][21]

'xx_username Genau der Piotoitsche Gelsenkirchener...wie gut das der Stempel noch frisch ist, dann kann man das toitsch sein sogar noch riechen...'

In [44]:
def max_from_labels(pred, labels):
    return labels[np.argmax(pred)].upper()

In [45]:
#submission['label'] = 'OTHER' # dummy label for fine classification
for index,row in submission.iterrows():
  submission['label'][index] = max_from_labels(preds[index], label_cols_fine)
  submission['fine'][index] = max_from_labels(preds[index], label_cols_fine)

//anaconda/envs/python3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
//anaconda/envs/python3/lib/python3.5/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
//anaconda/envs/python3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [46]:
# 3,6c -> 45.67
# 3,7c -> 43.8
# 3,6c -> 44.41 (1/2 classes)
# 3,6c -> 45.71* (2/1 classes)
# 3,6c -> 45.43 (3/1 classes)

# Balanced
# 4/1.0; 3,6c -> 45.72* (balanced) -> same lemmatized
submission.to_csv('naive2.csv', sep='\t', line_terminator='\n', header=None, index=False, encoding='utf-8-sig')

In [47]:
#for index,row in submission.iterrows():
#  print(preds[index])